In [1]:
from scipy.spatial import distance
from imutils import face_utils
import dlib
import cv2
import tensorflow as tf
import numpy as np
from playsound import playsound

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
LEFT_EYE = list(range(42, 48))
RIGHT_EYE = list(range(36, 42))

In [5]:
model = tf.keras.models.load_model("fatigue_detection_CNN.h5")

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

In [7]:
frame_counter = 0
init_ear_list = []
dynamic_threshold = None
EYE_AR_CONSEC_FRAMES = 20
INIT_FRAMES = 30
EAR_SCALE = 0.6  # EAR threshold is 60% of baseline EAR

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    status = "No Face Detected"
    color = (100, 100, 100)

    for face in faces:
        shape = predictor(gray, face)
        shape_np = np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)])

        left_eye = shape_np[LEFT_EYE]
        right_eye = shape_np[RIGHT_EYE]

        avg_EAR = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0

        cv2.polylines(frame, [left_eye.reshape(-1, 1, 2)], True, (0, 255, 0), 1)
        cv2.polylines(frame, [right_eye.reshape(-1, 1, 2)], True, (0, 255, 0), 1)

        if dynamic_threshold is None:
            if len(init_ear_list) < INIT_FRAMES:
                init_ear_list.append(avg_EAR)
                status = f"Collecting initial EAR... ({len(init_ear_list)}/{INIT_FRAMES})"
                color = (255, 255, 0)
            else:
                baseline_ear = np.mean(init_ear_list)
                dynamic_threshold = baseline_ear * EAR_SCALE
                print(f"[INFO] Dynamic EAR threshold set to {dynamic_threshold:.3f}")
                continue  # skip one frame before starting detection
        else:
            if avg_EAR < dynamic_threshold:
                frame_counter += 1
                if frame_counter >= EYE_AR_CONSEC_FRAMES:
                    status = "Fatigue Detected!"
                    color = (0, 0, 255)
                    # playsound("alarm.wav")
                else:
                    status = "Active"
                    color = (0, 255, 0)
            else:
                frame_counter = 0
                status = "Active"
                color = (0, 255, 0)
        
        break  # only process one face for now

    cv2.putText(frame, status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow("Fatigue Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[INFO] Dynamic EAR threshold set to 0.161
